# 03. Recommendations with Movie Subtitles vs. Descriptions

In [1]:
import pandas as pd
import numpy as np
import re
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction import _stop_words as stop_words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity as cosine

In [2]:
#Load the previously created dataset including the subtitles
df = pd.read_csv('data/netflix_with_subtitles.csv')
df

,title,listed_in,director,cast,description,subtitle
0,The Last Blockbuster,Documentaries,Taylor Morden,NaN,This nostalgic documentary reveals the real st...,Downloaded from YTS.MX Official YIFY movies si...
1,Franco Escamilla: Bienvenido al mundo,Stand-Up Comedy,Ulises Valencia,Franco Escamilla,Comedian Franco Escamilla shares stories about...,"It's a great honor to be here, to introduce yo..."
2,Last Knights,Action & Adventure,Kazuaki Kiriya,"Clive Owen, Morgan Freeman, Cliff Curtis, Akse...",A nobleman who values his people's well-being ...,"During the long, dark period of the Great Wars..."
3,Win It All,"Comedies, Independent Movies, Romantic Movies",Joe Swanberg,"Jake Johnson, Aislinn Derbez, Joe Lo Truglio, ...","After losing $50,000 that wasn't his, gambling...",Cubs parking! Easy entry and exit! How is it g...
4,Freedom Writers,Dramas,Richard LaGravenese,"Hilary Swank, Patrick Dempsey, Scott Glenn, Im...",While her at-risk students are reading classic...,There have been shots fired. Total civil unres...
...,...,...,...,...,...,...
1895,Fantastic Fungi,Documentaries,Louie Schwartzberg,NaN,"Delve into the magical world of fungi, from mu...",Subtitles by explosiveskull www.OpenSubtitles....
1896,The Zookeeper's Wife,Dramas,Niki Caro,"Jessica Chastain, Johan Heldenbergh, Daniel Br...","When the Nazis invade Poland, Warsaw Zoo caret...","ANTONINA: Hello, Jerzyk. JERZYK: Good morning,..."
1897,Frat Star,Comedies,"Grant S. Johnson, Ippsie Jones","Connor Lawrence, Justin Mark, Cathryn Dylan, C...",A freshman uninterested in joining a fraternit...,Hey! You piece of shit. We were having some fu...
1898,Star Men,Documentaries,Alison E. Rose,NaN,Four astronomers from England celebrate 50 yea...,subversive subs presents Star Wars: Episode I ...


## TIDF & Cosine Similarity

In [3]:
# I added some words like "i'm" or "he'll" to sklearn's stopword list as they were not included but I frequently spotted them in the subtitles
my_stop_words = stop_words.ENGLISH_STOP_WORDS.union(["i'm", "i'll", "i've", "you're", "you'll", "you've", "he's", "he'll", "she's", "she'll", "it's", "it'll", "we're", "we'll", "we've", "they're", "they'll", "they've", "can't", "couldn't", "hasn't", "haven't", "isn't", "aren't", "don't", "didn't", "shouldn't", "wouldn't", "won't"])

In [4]:
#Define Tokeniser
lem = WordNetLemmatizer()
def my_tokeniser(doc):
    tokens = re.split(r'[-\s.,;!?:()]+', doc)
    return [lem.lemmatize(t.lower()) for t in tokens if not t.lower() in my_stop_words]

In [5]:
#test the tokeniser -> looks good!
my_tokeniser(df.loc[1450]['subtitle'])

['perseverance',
 'important',
 'thing',
 'life',
 'believe',
 'want',
 'let',
 'anybody',
 'sidetrack',
 '1959',
 'le',
 'man',
 'race',
 'lap',
 'moss',
 'aston',
 'lead',
 'ferraris',
 'gendebien',
 'da',
 'silva',
 'ramos',
 'shelby',
 'ride',
 'tall',
 'man',
 'texas',
 'saddle',
 'long',
 'night',
 'drive',
 'solitary',
 'highspeed',
 'world',
 'sound',
 'broken',
 'routine',
 'stop',
 'refuel',
 'twentyfour',
 'hour',
 'long',
 'time',
 'lot',
 'luck',
 'pain',
 'chest',
 'knew',
 'wrong',
 'le',
 'man',
 'biggest',
 'race',
 'planet',
 'grueling',
 'race',
 'earth',
 'shelby',
 'condition',
 "that's",
 'secret',
 'team',
 'literally',
 'die',
 'moment',
 'leader',
 'return',
 'race',
 'forced',
 'drive',
 'nitroglycerin',
 'pill',
 'pain',
 'chest',
 "i'd",
 'pop',
 'slow',
 'couple',
 'lap',
 'deeper',
 '24hour',
 'race',
 "shelby's",
 'heart',
 'condition',
 'getting',
 'worse',
 'danger',
 'race',
 "doesn't",
 'heart',
 'raised',
 'southern',
 'baptist',
 'boy',
 "wasn't",
 

In [6]:
#TFIDF for subtitles
tfidf_vectoriser = TfidfVectorizer(tokenizer=my_tokeniser)
tfidf_subtitles = tfidf_vectoriser.fit_transform(df["subtitle"])
tfidf_subtitles.shape

/Users/sonja/opt/miniconda3/envs/perso/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(1900, 137980)

In [7]:
#Calculate cosine similarity between all movie subtitles
similarities_subtitles = cosine(tfidf_subtitles, tfidf_subtitles)
similarities_subtitles = pd.DataFrame(similarities_subtitles, columns = df["title"], index = df["title"])
similarities_subtitles.shape

(1900, 1900)

In [8]:
#TFIDF for descriptions
tfidf_descriptions = tfidf_vectoriser.fit_transform(df["description"])
tfidf_descriptions.shape

/Users/sonja/opt/miniconda3/envs/perso/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(1900, 7950)

In [9]:
#Calculate cosine similarity between all movie descriptions
similarities_descriptions = cosine(tfidf_descriptions, tfidf_descriptions)
similarities_descriptions = pd.DataFrame(similarities_descriptions, columns = df["title"], index = df["title"])
similarities_descriptions.shape

(1900, 1900)

## Get Recommendations

In [10]:
#function to get recommendations
pd.set_option("display.max_colwidth", None)
def get_recommendations(title, n, similarities):
    recommendation = similarities.sort_values(by = title, ascending=False)[title].index[1:n+1]
    df_recommendation = df.loc[df['title'].isin(recommendation)]
    df_recommendation = df_recommendation[['title', 'description', 'listed_in']]
    df_recommendation['title'] = pd.Categorical(df_recommendation['title'], recommendation)
    return df_recommendation.sort_values("title")

### For 'The Ivory Game'

In [11]:
#Infos about the movie
df.loc[df['title'] == 'The Ivory Game'][['title', 'description', 'listed_in']]

,title,description,listed_in
71,The Ivory Game,"Filmmakers infiltrate the corrupt global network of ivory trafficking, exposing poachers and dealers as African elephants edge closer to extinction.",Documentaries


In [12]:
#recommendations with subtitles
get_recommendations('The Ivory Game', 5, similarities_subtitles)

,title,description,listed_in
529,Naledi: A Baby Elephant's Tale,Scientists and animal keepers fight to save an orphaned baby elephant at a rescue camp in Botswana while struggling to defend its entire species.,Documentaries
884,Freshman Year,A devout preacher's son leaves his protective home to begin college and meets a self-reliant student who will test his faith and his heart.,"Dramas, Faith & Spirituality"
1609,Hostage House,"A realtor and her daughter get taken captive by two fugitive criminals during an open house, forcing her to find creative ways to ensure their survival.",Thrillers
9,Obvious Child,"After being fired from her job and dumped by her cheating boyfriend, a comedian has a one-night stand. Weeks later, she finds out she's pregnant.","Comedies, Dramas, Independent Movies"
207,The World We Make,"A teenage equestrian and a local football player fall for each other, but simmering racism in their small town puts their relationship to the test.","Dramas, Romantic Movies"


In [13]:
#recommendations with descriptions
get_recommendations('The Ivory Game', 5, similarities_descriptions)

,title,description,listed_in
94,Asura: The City of Madness,"Caught between a corrupt mayor and a prosecutor intent on exposing political depravity, a police detective must decide whose side he's on.","Action & Adventure, Dramas, International Movies"
354,1st Summoning,"Student filmmakers uncover occult rituals tied to an abandoned warehouse, then gradually realize the horror is closer than they think.","Horror Movies, Independent Movies"
1310,Seaspiracy,"Passionate about ocean life, a filmmaker sets out to document the harm that humans do to marine species — and uncovers alarming global corruption.",Documentaries
1474,The Seven Five,"This riveting film examines the case of Mike Dowd, a corrupt Brooklyn cop who became dirtier than the drug dealers he was supposed to be policing.",Documentaries
303,Deewana Main Deewana,A man's instant attraction to a woman becomes an obsession as he adopts a friend's identity and even commits murder in order to get closer to her.,"Comedies, Dramas, International Movies"


**Subtitles**

The first recommendation with the subtitles ('Naledi: A Baby Elephant's Tale) is definitely the best - both films are documentaries about elephants. The other films, however, do not seem to be good recommendations. Since 'The Ivory Game' is about the illegal hunting of elephants, I can imagine that the subtitles contain many words similar to those in the thriller 'Hostage House' (third recommendation).

**Descriptions**

"Seaspiracy" in the third place seems like a good match, as it is also a nature documentary about the harm humans can do to animals. The rest of the movies don't seem like good recommendations either.

### For 'Sherlock Holmes'

In [14]:
df.loc[df['title'] == 'Sherlock Holmes'][['title', 'description', 'listed_in']]

,title,description,listed_in
956,Sherlock Holmes,The game is afoot for an eccentric detective who must use his brains and brawn to track down a nefarious nemesis with the help of his partner.,"Action & Adventure, Comedies"


In [15]:
get_recommendations('Sherlock Holmes', 5, similarities_subtitles)

,title,description,listed_in
1575,Mona Lisa Smile,"In 1953, the women of Wellesley College are measured by how well they marry – until the arrival of a professor who threatens to upend the status quo.",Dramas
118,Gosford Park,"In this witty whodunit, wealthy guests and their servants gather for a weekend stay at the home of Sir William McCordle, who is soon found murdered.","Comedies, Dramas, International Movies"
763,Special 26,"In this fact-based crime drama, a gang of con artists posing as government officials executes a heist in the form of a raid on a prominent jeweler.","Dramas, International Movies, Thrillers"
639,Love Dot Com: The Social Experiment,"Sparks fly and romance blooms when Shelby, a vegan chef, meets Greg, a developer – until their opposing views clash.","Comedies, Romantic Movies"
336,SGT. Will Gardner,"A homeless vet who has PTSD steals a motorcycle and hits the road, trying to sort out what life means now that war is in the rear-view mirror.",Dramas


In [16]:
get_recommendations('Sherlock Holmes', 5, similarities_descriptions)

,title,description,listed_in
305,Sprinter,"When a Jamaican teen hopes to use track to reunite him with his mother in America, family dysfunction at home threatens to knock his plans off course.","Dramas, International Movies, Sports Movies"
392,Truth or Dare,"A game of ""Truth or Dare"" among college friends on spring break turns deadly when a demon seems determined to use it to kill them all.",Horror Movies
1009,P.S. I Love You,"When she loses her beloved husband to a brain tumor, grieving widow Holly Kennedy learns that he left a series of letters behind to help her cope.","Dramas, Romantic Movies"
74,Escape from Mr. Lemoncello’s Library,An eccentric game designer builds a library full of traps and tricks for a group of kids who score the chance to navigate his elaborate scavenger hunt.,Children & Family Movies
148,One 2 Ka 4,"When his partner is killed, a young cop is forced to care for the man's four children, and solicits the help of a woman who moves in with them.","Action & Adventure, Comedies, Dramas"


The first three recommendations with subtitles seem quite good, the fourth and fifth do not. Overall, they appear better than the recommendations with descriptions.

### For 'Penguins of Madagascar: The Movie'

In [17]:
get_recommendations('Penguins of Madagascar: The Movie', 5, similarities_subtitles)

,title,description,listed_in
1592,Anger Management,"After a gentle businessman is forced to get anger management counseling, his therapist moves in – but turns out to have anger issues of his own.",Comedies
1435,Madagascar 3: Europe's Most Wanted,"Next stop: New York! But to get back home, Alex and friends must hitch a ride with a traveling European circus and evade an evil animal-control officer.","Children & Family Movies, Comedies"
1658,Band Aid,"A young married couple tries to save their disintegrating relationship by turning their fights into songs, with the help of an oddball neighbor.","Comedies, Dramas, Independent Movies"
247,Penguin Bloom,"As a mom copes with the aftermath of a harrowing accident, she finds inspiration from an injured magpie taken in by her family. Based on a true story.","Children & Family Movies, Dramas"
923,The End of the Tour,"""Rolling Stone"" reporter David Lipsky shadows famed writer David Foster Wallace in this fictionalized account of their revealing five-day encounter.","Dramas, Independent Movies"


In [18]:
get_recommendations('Penguins of Madagascar: The Movie', 5, similarities_descriptions)

,title,description,listed_in
1873,Hot Property,"When a resourceful but flighty Londoner is threatened with eviction, she vows to do whatever it takes to defeat a greedy real estate agent.","Comedies, Independent Movies"
816,How to Train Your Dragon 2,"Hiccup, Toothless and a mysterious dragon rider join forces to protect the island of Berk from a power-hungry warrior with a dragon army of his own.",Children & Family Movies
729,Act of Valor,An elite squad of Navy SEALs is tasked with rescuing a kidnapped CIA agent from a lethal terrorist cell.,Action & Adventure
420,Devil's Gate,"Seeking a missing woman in North Dakota, an FBI agent and a sheriff focus on her religious zealot husband but discover something far more sinister.","Horror Movies, Sci-Fi & Fantasy, Thrillers"
302,Chamatkar,"A village schoolteacher and the ghost of a murdered man, who were both betrayed by those closest to them, join forces to help each other.","Comedies, International Movies, Sci-Fi & Fantasy"


In [19]:
df.loc[df['title'] == 'Penguins of Madagascar: The Movie'][['title', 'description', 'listed_in']]

,title,description,listed_in
32,Penguins of Madagascar: The Movie,"Elite penguin spies Skipper, Kowalski, Rico and Private join forces with the suave agents of the North Wind to defeat power-mad genius Octavius Brine.","Children & Family Movies, Comedies"


**Subtitles**

It spotted the similarities with 'Madagascar 3: Europe's Most Wanted'. However, the other recommendations don't seem great.

**Descriptions**

'How to Train Your Dragon 2' is a great recommendation, the others don't seem to be as good.

_____
## Combining the recommendations with other properties (director, cast, genre)
Let's go trough the same process as in the first notebook: Creating bag-of-words for the other features and then combine it with the cosine similarities of the subtitles / descriptions.

### Bag-of-Words and cosine similarity for director, cast & genre

In [20]:
#replace NaN values in df with empty strings
df_new = df.fillna('')
df_new = df_new[['title', 'listed_in', 'director', 'cast']]

In [21]:
## The code in this cell is from Niharika Pandit (2021), 'Netflix Visualizations, Recommendation, EDA', Kaggle, link: https://www.kaggle.com/code/niharika41298/netflix-visualizations-recommendation-eda#Recommendation-System-(Content-Based)
## slightly modified to fit my selected features

#Make all parameters lowercase & remove spaces between words 
# -> e.g. Vanessa Hudgens becomes vanessahudgens and Vannessa Kirby becomes vannessakirby so that their first name is not registered as the same word
def clean_data(x):
        return str.lower(x.replace(" ", "").replace(","," ")) #modified this line to also replace commas with spaces
features=['listed_in','director','cast']
for feature in features:
    df_new[feature] = df_new[feature].apply(clean_data)
    
df_new.head(3)

,title,listed_in,director,cast
0,The Last Blockbuster,documentaries,taylormorden,
1,Franco Escamilla: Bienvenido al mundo,stand-upcomedy,ulisesvalencia,francoescamilla
2,Last Knights,action&adventure,kazuakikiriya,cliveowen morganfreeman cliffcurtis akselhennie peymanmoaadi ayeletzurer shohrehaghdashloo tsuyoshiihara ahnsung-kee


In [22]:
## The code in this cell is from Niharika Pandit (2021), 'Netflix Visualizations, Recommendation, EDA', Kaggle, link: https://www.kaggle.com/code/niharika41298/netflix-visualizations-recommendation-eda#Recommendation-System-(Content-Based)
from sklearn.feature_extraction.text import CountVectorizer
def combined(x):
    return x['director'] + ' ' + x['cast'] + ' ' +x['listed_in']

df_new['combined'] = df_new.apply(combined, axis=1)

#Using CountVectorizer to get bag of words
vectoriser = CountVectorizer(stop_words='english')
bag_of_words = vectoriser.fit_transform(df_new['combined'])
bag_of_words.shape

(1900, 13035)

In [23]:
#Calculate cosine similarity
similarities_features = cosine(bag_of_words)
similarities_features = pd.DataFrame(similarities_features, columns = df["title"], index = df["title"])
similarities_features.shape

(1900, 1900)

### Combine & Get Recommendations
I wanted to see how the recommendations turned out when we combine the features with subtitles compared to the descriptions, but also whether it would improve the results if we combined all three (features + subtitles + descriptions).

In [24]:
text_weight = 2 #control the weight of the subtitles / descriptions 

#Combine cosine similarity dataframes
create_sum = lambda s1, s2: s1 + s2#
#features + subtitles
similarities_features_subtitles = similarities_features.combine((similarities_subtitles*text_weight), create_sum)
#features + descriptions
similarities_features_descriptions = similarities_features.combine((similarities_descriptions*text_weight), create_sum)
#features + subtitles + descriptions
total_similarities = similarities_features_subtitles.combine(similarities_descriptions*text_weight, create_sum)

#### For 'The Ivory Game'

In [25]:
#features + subtitles
get_recommendations('The Ivory Game', 5, similarities_features_subtitles)

,title,description,listed_in
529,Naledi: A Baby Elephant's Tale,Scientists and animal keepers fight to save an orphaned baby elephant at a rescue camp in Botswana while struggling to defend its entire species.,Documentaries
907,The Minimalists: Less Is Now,They've built a movement out of minimalism. Longtime friends Joshua Fields Millburn and Ryan Nicodemus share how our lives can be better with less.,Documentaries
453,Bending the Arc,This documentary follows a group of ambitious advocates whose mission to save lives in Haiti turns into a global fight for health care and justice.,Documentaries
1336,Murder to Mercy: The Cyntoia Brown Story,"After 16-year-old Cyntoia Brown is sentenced to life in prison, questions about her past, physiology and the law itself call her guilt into question.",Documentaries
1197,Creating The Queen's Gambit,"A fascinating character. Exquisite sets. A wig for every era. The stars, creators and crew reveal how the hit series about a chess prodigy came to life.",Documentaries


In [26]:
#features + descriptions
get_recommendations('The Ivory Game', 5, similarities_features_descriptions)

,title,description,listed_in
453,Bending the Arc,This documentary follows a group of ambitious advocates whose mission to save lives in Haiti turns into a global fight for health care and justice.,Documentaries
1310,Seaspiracy,"Passionate about ocean life, a filmmaker sets out to document the harm that humans do to marine species — and uncovers alarming global corruption.",Documentaries
1474,The Seven Five,"This riveting film examines the case of Mike Dowd, a corrupt Brooklyn cop who became dirtier than the drug dealers he was supposed to be policing.",Documentaries
907,The Minimalists: Less Is Now,They've built a movement out of minimalism. Longtime friends Joshua Fields Millburn and Ryan Nicodemus share how our lives can be better with less.,Documentaries
1336,Murder to Mercy: The Cyntoia Brown Story,"After 16-year-old Cyntoia Brown is sentenced to life in prison, questions about her past, physiology and the law itself call her guilt into question.",Documentaries


As we could also observe in the first notebook, the recommendations get a bit better when also using the features. However, "Naledi: A Baby Elephant's Tale" is still the best recommendation overall.

In [27]:
#features + subtitles + descriptions
get_recommendations('The Ivory Game', 5, total_similarities)

,title,description,listed_in
529,Naledi: A Baby Elephant's Tale,Scientists and animal keepers fight to save an orphaned baby elephant at a rescue camp in Botswana while struggling to defend its entire species.,Documentaries
453,Bending the Arc,This documentary follows a group of ambitious advocates whose mission to save lives in Haiti turns into a global fight for health care and justice.,Documentaries
907,The Minimalists: Less Is Now,They've built a movement out of minimalism. Longtime friends Joshua Fields Millburn and Ryan Nicodemus share how our lives can be better with less.,Documentaries
1310,Seaspiracy,"Passionate about ocean life, a filmmaker sets out to document the harm that humans do to marine species — and uncovers alarming global corruption.",Documentaries
1474,The Seven Five,"This riveting film examines the case of Mike Dowd, a corrupt Brooklyn cop who became dirtier than the drug dealers he was supposed to be policing.",Documentaries


As you might expect, it is a mixture of the two previous results. Nevertheless, I like the fact that "Naledi: A Baby Elephant's Tale" and "Seaspiracy", the two best recommendations, are both in this output.

#### For 'Sherlock Holmes'

In [28]:
get_recommendations('Sherlock Holmes', 5, similarities_features_subtitles)

,title,description,listed_in
122,Due Date,"Days before his pregnant wife's due date, Peter lands on the ""no-fly"" list, forcing him to drive across the country with an irritating slacker.","Action & Adventure, Comedies"
1607,Polar,"An assassin on the verge of retirement must put the good life on hold when his greedy boss sends a squad of young, ruthless killers to take him out.","Action & Adventure, International Movies"
805,"Lock, Stock and Two Smoking Barrels","Hoping to make a bundle in a high-stakes poker game, four shiftless lads from London's East End instead find themselves swimming in debt.","Action & Adventure, Comedies, Independent Movies"
877,Otherhood,"Feeling forgotten on Mother's Day, three best friends leave the suburbs and drive to New York City to surprise their adult sons.",Comedies
1566,30 Minutes or Less,Two crooks planning a bank heist wind up abducting a pizza delivery driver and force him to commit the robbery — with a strict time limit.,"Action & Adventure, Comedies"


In [29]:
get_recommendations('Sherlock Holmes', 5, similarities_features_descriptions)

,title,description,listed_in
1143,The Happytime Murders,"In an LA where puppets and humans coexist, a down-on-his-luck detective teams with his ex-partner to nab a killer targeting the cast of an old TV show.","Action & Adventure, Comedies"
148,One 2 Ka 4,"When his partner is killed, a young cop is forced to care for the man's four children, and solicits the help of a woman who moves in with them.","Action & Adventure, Comedies, Dramas"
855,Bulletproof 2,A special agent abruptly reunites with a criminal – once a former friend – when he attempts to use his identity to infiltrate and take down a cartel.,"Action & Adventure, Comedies"
805,"Lock, Stock and Two Smoking Barrels","Hoping to make a bundle in a high-stakes poker game, four shiftless lads from London's East End instead find themselves swimming in debt.","Action & Adventure, Comedies, Independent Movies"
1749,National Lampoon's Loaded Weapon 1,"In this parody of the Lethal Weapon franchise, criminals murder a detective and steal her prized evidence: microfilm with a recipe for cocaine cookies. Luckily, loose-cannon cop Jack Colt and his by-the-book partner, Wes Luger, are on the case.","Action & Adventure, Comedies, Independent Movies"


The recommendations with subtitles deteriorated - without the features they were much better. The results with the descriptions, on the other hand, got better.

In [30]:
get_recommendations('Sherlock Holmes', 5, total_similarities)

,title,description,listed_in
148,One 2 Ka 4,"When his partner is killed, a young cop is forced to care for the man's four children, and solicits the help of a woman who moves in with them.","Action & Adventure, Comedies, Dramas"
1143,The Happytime Murders,"In an LA where puppets and humans coexist, a down-on-his-luck detective teams with his ex-partner to nab a killer targeting the cast of an old TV show.","Action & Adventure, Comedies"
805,"Lock, Stock and Two Smoking Barrels","Hoping to make a bundle in a high-stakes poker game, four shiftless lads from London's East End instead find themselves swimming in debt.","Action & Adventure, Comedies, Independent Movies"
855,Bulletproof 2,A special agent abruptly reunites with a criminal – once a former friend – when he attempts to use his identity to infiltrate and take down a cartel.,"Action & Adventure, Comedies"
1289,The Assignment,A hitman has his body surgically altered into a woman's against his will. Now he has to track down the surgeon responsible and get his revenge.,Action & Adventure


Interestingly, we can see that it uses many movies that were also recommended with the descriptions & features.

#### For 'Penguins of Madagascar'

In [31]:
get_recommendations('Penguins of Madagascar: The Movie', 5, similarities_features_subtitles)

,title,description,listed_in
1435,Madagascar 3: Europe's Most Wanted,"Next stop: New York! But to get back home, Alex and friends must hitch a ride with a traveling European circus and evade an evil animal-control officer.","Children & Family Movies, Comedies"
344,Keymon and Nani in Space Adventure,"For Rohan and his magical pal, Keymon, a trip to visit their grandmother turns into the rescue mission of a lifetime when she is kidnapped by aliens!","Children & Family Movies, Comedies"
1592,Anger Management,"After a gentle businessman is forced to get anger management counseling, his therapist moves in – but turns out to have anger issues of his own.",Comedies
247,Penguin Bloom,"As a mom copes with the aftermath of a harrowing accident, she finds inspiration from an injured magpie taken in by her family. Based on a true story.","Children & Family Movies, Dramas"
877,Otherhood,"Feeling forgotten on Mother's Day, three best friends leave the suburbs and drive to New York City to surprise their adult sons.",Comedies


In [34]:
get_recommendations('Penguins of Madagascar: The Movie', 5, similarities_features_descriptions)

,title,description,listed_in
344,Keymon and Nani in Space Adventure,"For Rohan and his magical pal, Keymon, a trip to visit their grandmother turns into the rescue mission of a lifetime when she is kidnapped by aliens!","Children & Family Movies, Comedies"
1435,Madagascar 3: Europe's Most Wanted,"Next stop: New York! But to get back home, Alex and friends must hitch a ride with a traveling European circus and evade an evil animal-control officer.","Children & Family Movies, Comedies"
839,Super Monsters: Dia de los Monsters,"The Super Monsters celebrate Día de los Muertos in Vida’s hometown with her magical family, some new monster friends and a spook-tacular parade!",Children & Family Movies
268,Pets United,A street-smart dog and a pampered cat join forces to lead a pack of unlikely heroes when their city is seized by its evil mayor — and his robot army.,"Children & Family Movies, Comedies"
472,Spy Kids 2: The Island of Lost Dreams,"Spy kids Carmen and Juni Cortez team up with two other pint-size secret agents, Gary and Gerti Giggles, in order to take on an evil scientist.","Children & Family Movies, Comedies"


In [35]:
get_recommendations('Penguins of Madagascar: The Movie', 5, total_similarities)

,title,description,listed_in
1435,Madagascar 3: Europe's Most Wanted,"Next stop: New York! But to get back home, Alex and friends must hitch a ride with a traveling European circus and evade an evil animal-control officer.","Children & Family Movies, Comedies"
344,Keymon and Nani in Space Adventure,"For Rohan and his magical pal, Keymon, a trip to visit their grandmother turns into the rescue mission of a lifetime when she is kidnapped by aliens!","Children & Family Movies, Comedies"
929,The Sleepover,What do you do if your parents are kidnapped by a crew of international thieves? You begin a wild overnight adventure — complete with spy gear.,"Action & Adventure, Children & Family Movies, Comedies"
1592,Anger Management,"After a gentle businessman is forced to get anger management counseling, his therapist moves in – but turns out to have anger issues of his own.",Comedies
64,Surf's Up,This Oscar-nominated animated comedy goes behind the scenes at the Penguin World Surfing Championship to get a close-up look at the inventors of surfing.,"Children & Family Movies, Comedies, Sports Movies"


In this case, the recommendations generated with descriptions + features actually seem like the best ones.

## Conclusion
It is difficult to say how the techniques compare. 
- Looking only at the descriptions compared to the subtitles, the use of subtitles might lead to slightly better recommendations. 
- When other features were added (genre, director and cast), the results varied greatly. Sometimes the descriptions + features even gave better results.

It might be useful to use longer summaries of the films instead of the descriptions or subtitles. These could be taken from Wikipedia, for example. This would leave less room for errors, as we have seen with the very short descriptions in the first notebook. At the same time, this could lead to better results than using subtitles, as the plot is better summarised and also includes actions instead of just spoken words.